## Critical Sections: The Synchronization Problem

We are now going to take a deeper look inside synchronization of threads/processes and the principles needed to share data safely. There are two typical goals:
* Contention: 
  * How to resolve the conflicts that result from multiple processes trying to access shared resources?
* Cooperation:
  * An action by one process may enable another action by another process
  * In such cases, processes should coordinate their actions
  
Maybe do the parable of buying milk to motivate why it is hard....... Or find a better example.

Synchronization issues are the major bug in parallel programs
* Deadlock: when a program cannot progress because all of it's threads/processes are waiting on resources.
* Incorrect results: from uncontrolled sharing (race conditions) on data.

The constructs/algorithms underlying critical sections (locks, atomic variables) are complex.
Understanding them will help you use them well.

  
### Mutual Exclusion

Mutual exclusion between is the core problem in synchronization.
It guarantees
  * Exclusive access to a shared resource among competing processes
  * No deadlock
  * Starvation resistance (must make progress eventually)

__Peterson's Algorithm__ gives a solution for two processes.

```python
# process 0
b[0] = True
turn = 0
await(b[1]==False or turn==0)
# critical section
...
b[0] = False
```

```python
# process 1
b[1] = True
turn = 1
await(b[0]==False or turn==1)
# critical section
...
b[1] = False
```

* b[x] indicates process b’s desire for resource x
* Write to turn will resolve who got there first
  * concurrent writes to `turn` can race against each other the variable will be assigned by the later write
  * the last writer gives the mutex to the other party 
* `await` ensures that either:
  * the other party is not contending
  * or the other party wrote `turn` to give precedence
  
* Peterson's algorithms has the following properties
  * mutual exclusion (of the critical section code)
  * _starvation resistant_: no process can got twice in a row while the other process is waiting
  * contention free overhead of 4 memory accesses.  For process 0
    1. write `b[0]`
    2. write `turn`
    3. read `b[1]`
    4. write `b[0]
  * Uses three shared _atomic_ registers
      * atomic here means atomic read and atomic write
  * 
    
    


